In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
real_dir = '/content/drive/My Drive/dspdataset/real'
fake_dir = '/content/drive/My Drive/dspdataset/fake'

In [3]:
import os

print("Real Images:", os.listdir(real_dir)[:5])  # Display first 5 images in real folder
print("Fake Images:", os.listdir(fake_dir)[:5])  # Display first 5 images in fake folder


Real Images: ['real_00002.jpg', 'real_00005.jpg', 'real_00349.jpg', 'real_00003.jpg', 'real_00350.jpg']
Fake Images: ['easy_98_0001.jpg', 'easy_99_0101.jpg', 'easy_12_1110.jpg', 'easy_4_0011.jpg', 'easy_10_0001.jpg']


In [4]:
# Import necessary libraries
import os
import numpy as np
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.models import Model
from google.colab import drive

In [5]:
IMG_SIZE = 128  # Resized image dimensions

# Load images
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)  # Read image
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize image to 128x128
            images.append(img)
            labels.append(label)  # 0 for real, 1 for fake
    return np.array(images), np.array(labels)

In [6]:
# Load images and labels
real_images, real_labels = load_images_from_folder(real_dir, label=0)
fake_images, fake_labels = load_images_from_folder(fake_dir, label=1)

# Combine images and labels
X = np.concatenate((real_images, fake_images), axis=0)
y = np.concatenate((real_labels, fake_labels), axis=0)

# Shuffle dataset
X, y = shuffle(X, y, random_state=42)

# Normalize images to [0, 1]
X = X / 255.0

# Display dataset information
print(f"Total images loaded: {len(X)}")
print(f"Shape of images: {X.shape}")  # Expected shape: (num_samples, 128, 128, 3)
print(f"Shape of labels: {y.shape}")

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Total images loaded: 660
Shape of images: (660, 128, 128, 3)
Shape of labels: (660,)


In [7]:
# Initialize ResNet50 for feature extraction
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
feature_extractor = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)  # Remove last layer


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [8]:
def extract_features(data):
    data = preprocess_input(data)  # Preprocess for ResNet
    features = feature_extractor.predict(data)
    return features.reshape(features.shape[0], -1)  # Flatten the features

In [9]:
# Extract features using ResNet
X_train_features = extract_features(X_train)
X_test_features = extract_features(X_test)


17/17 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step


In [10]:
print(f"Shape of extracted train features: {X_train_features.shape}")
print(f"Shape of extracted test features: {X_test_features.shape}")

Shape of extracted train features: (528, 32768)
Shape of extracted test features: (132, 32768)


In [11]:
# Train Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_features, y_train)

# Predict on test set
y_pred = clf.predict(X_test_features)

In [12]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.61
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.70      0.67        76
           1       0.54      0.48      0.51        56

    accuracy                           0.61       132
   macro avg       0.59      0.59      0.59       132
weighted avg       0.60      0.61      0.60       132

Confusion Matrix:
 [[53 23]
 [29 27]]
